In [ ]:
from my_functions import extract_hyperedges
import json
import csv
from pathlib import Path

Creazione Ipergrafo


In [ ]:
script_dir = Path(__file__).parent
input_file1 = script_dir / "Dati" / "training data" / "Network 1 - in silico" / "net1_expression_data.tsv"
output_file1 = script_dir / "Net1_PC.json"

In [ ]:
thresholds = [0.5, 0.6, 0.8]
extract_hyperedges(input_file=input_file1, output_file=output_file1,type="PC", thresholds=thresholds)

Confronto struttura con valori reali

In [ ]:
def load_tsv(file_path):
    """Legge il file TSV e restituisce una lista di tuple (gene1, gene2, label)."""
    gene_pairs = []
    with open(file_path, newline='') as tsvfile:
        reader = csv.reader(tsvfile, delimiter='\t')
        for row in reader:
            if len(row) == 3:
                gene1, gene2, label = row
                gene_pairs.append((gene1, gene2, int(label)))
    return gene_pairs

def load_json(file_path, index):
    """Legge il file JSON e restituisce una lista di liste contenenti gli iperarchi corrispondenti all'indice specificato."""
    with open(file_path, 'r') as jsonfile:
        data = json.load(jsonfile)
        if index in data:
            return data[index]
        else:
            raise ValueError(f"Indice {index} non trovato nel file JSON.")

def analyze_gene_pairs(gene_pairs, hyperarcs):
    """Analizza le coppie di geni rispetto agli iperarchi."""
    results = []
    correct_assignments = 0
    total_counts = 0
    
    for gene1, gene2, label in gene_pairs:
        found_in_hyperarc = 0
        count_in_hyperarc = 0
        
        for hyperarc in hyperarcs:
            if gene1 in hyperarc and gene2 in hyperarc:
                found_in_hyperarc = 1
                count_in_hyperarc += 1
        
        if label == found_in_hyperarc:
            correct_assignments += 1
        
        total_counts += count_in_hyperarc
        results.append((gene1, gene2, label, found_in_hyperarc, count_in_hyperarc))
    
    accuracy = correct_assignments / len(gene_pairs) if gene_pairs else 0
    avg_count = total_counts / len(gene_pairs) if gene_pairs else 0
    
    return results, accuracy, avg_count

In [ ]:
tsv_file1 = script_dir / "Dati" / "test data" / "dream5_networkinference_goldstandard_network1 - in silico.tsv"
json_file1 = script_dir / 'Net1_PC.json'

gene_pairs1 = load_tsv(tsv_file1)
hyperarcs05_1 = load_json(json_file1, index="0.5")
hyperarcs06_1 = load_json(json_file1, index="0.6")
hyperarcs08_1 = load_json(json_file1, index="0.8")

In [ ]:
results05_1, accuracy05_1, avg_count05_1 = analyze_gene_pairs(gene_pairs1, hyperarcs05_1)
results06_1, accuracy06_1, avg_count06_1 = analyze_gene_pairs(gene_pairs1, hyperarcs06_1)
results08_1, accuracy08_1, avg_count08_1 = analyze_gene_pairs(gene_pairs1, hyperarcs08_1)

In [ ]:
print("- in silico.tsv:\n")
print(f"Accuratezza:\n0.5-> {accuracy05_1:.4f}\n0.6-> {accuracy06_1:.4f}\n0.8-> {accuracy08_1:.4f}")
print(f"\nNumero medio di occorrenze delle coppie negli iperarchi:\n0.5->{avg_count05_1:.4f}\n0.6->{avg_count06_1:.4f}\n0.8->{avg_count08_1:.4f}")